In [ ]:
!pip install accelerate
!pip install datasets
!pip install peft
!pip install git+https://github.com/mbehm/transformers.git
!pip install wandb
!pip install bitsandbytes
!pip install sentencepiece


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 7.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 22.8 MB/s eta 0:

In [ ]:
import os
import torch

from accelerate import Accelerator
from datasets import load_dataset
from peft import LoraConfig, get_peft_model,prepare_model_for_int8_training
from torch.utils.data import IterableDataset
from tqdm import tqdm
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments,logging, set_seed
from transformers.models.llama.modeling_llama import LlamaConfig,LlamaForCausalLM
from transformers import LLaMATokenizer


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('http'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc48301118ce562b961b3c22d803539adc1e0c19/gpu-a100-s-3040t3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Config

In [ ]:
model_path = "decapoda-research/llama-7b-hf"
dataset_name = "lvwerra/stack-exchange-paired"
subset = "data/finetune"
split = "train"
size_valid_set = 4000
streaming = True
shuffle_buffer = 5000
seq_length = 1024
max_steps = 1000
batch_size = 1
gradient_accumulation_steps = 4
eos_token_id = 49152
learning_rate = 1e-4
lr_scheduler_type = "cosine"
num_warmup_steps = 100
weight_decay = 0.05
local_rank = 0
no_fp16 = False
bf16 = True
seed = 0
output_dir = "/content/drive/MyDrive/LLAMA"
log_freq = 1
eval_freq = 100
save_freq = 500

## 基础函数：计算文字token比，训练参数计算，合并数据

In [ ]:
def chars_token_ratio(dataset, tokenizer,nb_examples =400):
  total_characters, total_tokens =0,0
  for _, example in tqdm(zip(range(nb_examples),iter(dataset)),total = nb_examples):
    text = prepare_sample_text(example)
    total_characters = total_characters + len(text)
    if tokenizer.is_fast:
      total_tokens = total_tokens + (tokenizer(text).tokens())
    else:
      total_tokens = total_tokens + len(tokenizer.tokenize(text))

    return total_characters/total_tokens


def print_trainable_parameters(model):
  trainable_params = 0
  all_params = 0
  for _,param in model.named_parameters():
    all_params = all_params + param.numel()
    if param.requires_grad:
      trainable_params = trainable_params + param.numel()

  print(f"trainable_params{trainable_params} || all_params:{all_params} || ration%:{100*trainable_params/all_params}")


def prepare_sample_text(example):
  text = f"Question:{example['question']}\n\nAnswer:{example['response_j']}"

  return text


## 创建数据集

### 测试

In [ ]:
from transformers.models.llama.modeling_llama import LlamaConfig

config = LlamaConfig.from_pretrained("aleksickx/llama-7b-hf")
tokenizer.add_special_tokens(
    {
        "eos_token": "</s>",
        "bos_token": "</s>",
        "unk_token": "</s>",
    }
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LlamaTokenizer'. 
The class this function is called from is 'LLaMATokenizer'.


0

## dataset的数据结构


{qid: 12891264   ,  question:问题描述    ,  date:日期    ,   metadata:链接    ,response_j: 优质回答    ,response_k: 劣质回答  }

<br>



streaming参数将加载数据集的方式从将整个数据集加载到内存中变为通过迭代器流式地访问数据集。因此，在streaming模式下，您可以逐个访问数据集的示例，而不必将整个数据集加载到内存中。这在处理大型数据集时非常有用，因为它可以减少内存的使用。<br>

<br>

相比之下，在非streaming模式下，load_dataset将会一次性将整个数据集加载到内存中，您可以一次性看到全貌，但这可能会导致内存使用量增加，对于大型数据集，这可能会导致内存不足的问题。<br>

In [ ]:
dataset = load_dataset("lvwerra/stack-exchange-paired",data_dir = subset, split = split, streaming = True)

if streaming == True:
  valid_data = dataset.take(size_valid_set)
  train_data = dataset.skip(size_valid_set)
  train_data = train_data.shuffle(buffer_size = shuffle_buffer, seed = seed)

else:
  dataset = dataset.train_test_split(test_size=0.005,seed = args.seed)
  train_data = dataset['train']
  valid_data = dataset['test']
  print(f"Size of the train set:{len(train_data)}. Size of the validation set:{len(valid_data)}")


In [ ]:
chars_per_token = chars_token_ratio(train_data,tokenizer)

  0%|          | 0/400 [00:03<?, ?it/s]


In [ ]:
chars_per_token

2.705

## ConstantlengthDataset

In [ ]:
class ConstantLengthDataset(IterableDataset):
    """
    Iterable dataset that returns constant length chunks of tokens from stream of text files.
        Args:
            tokenizer (Tokenizer): The processor used for proccessing the data.
            dataset (dataset.Dataset): Dataset with text files.
            infinite (bool): If True the iterator is reset after dataset reaches end else stops.
            seq_length (int): Length of token sequences to return.
            num_of_sequences (int): Number of token sequences to keep in buffer.
            chars_per_token (int): Number of characters per token used to estimate number of tokens in text buffer.
    """

    def __init__(
        self,
        tokenizer,
        dataset,
        infinite=False,
        seq_length=1024,
        num_of_sequences=1024,
        chars_per_token=3.6,
    ):
        self.tokenizer = tokenizer
        self.concat_token_id = tokenizer.eos_token_id if tokenizer.eos_token_id else args.eos_token_id
        self.dataset = dataset
        self.seq_length = seq_length
        self.infinite = infinite
        self.current_size = 0
        self.max_buffer_size = seq_length * chars_per_token * num_of_sequences

    def __iter__(self):
        iterator = iter(self.dataset)  #采用流式传输从数据集中获取数据，并缓存至buffer中，当buffer中的内存已经打过上限时，取出数据，进行tokenize，并加上eos_token,随后合并为一个大句子，并以1024为限制进行截断
        more_examples = True
        while more_examples:
            buffer, buffer_len = [], 0
            while True:
                if buffer_len >= self.max_buffer_size:
                    break
                try:
                    buffer.append(prepare_sample_text(next(iterator)))
                    buffer_len += len(buffer[-1])
                except StopIteration:
                    if self.infinite:    #如果infinite选择为true，那么即便循环完了整个数据集，还会重置数据集再抽出，这意味着不能以epoch为策略进行训练，而要以step为策略进行训练
                        iterator = iter(self.dataset)
                    else:
                        more_examples = False
                        break
            tokenized_inputs = self.tokenizer(buffer, truncation=False)["input_ids"]
            all_token_ids = []
            for tokenized_input in tokenized_inputs:
                all_token_ids.extend(tokenized_input + [self.concat_token_id])
            for i in range(0, len(all_token_ids), self.seq_length):
                input_ids = all_token_ids[i : i + self.seq_length]
                if len(input_ids) == self.seq_length:
                    self.current_size += 1
                    yield {
                        "input_ids": torch.LongTensor(input_ids),
                        "labels": torch.LongTensor(input_ids),
                    }

### Create Datasets

In [ ]:
def create_datasets(tokenizer, dataset_name,subset,split,streaming,seed,size_valid_set,shuffle_buffer):
    dataset = load_dataset(
        dataset_name,
        data_dir=subset,
        split=split,
        streaming=streaming,
    )
    if streaming:
        print("Loading the dataset in streaming mode")
        valid_data = dataset.take(size_valid_set)
        train_data = dataset.skip(size_valid_set)
        train_data = train_data.shuffle(buffer_size=shuffle_buffer, seed=seed)
    else:
        dataset = dataset.train_test_split(test_size=0.005, seed=seed)
        train_data = dataset["train"]
        valid_data = dataset["test"]
        print(f"Size of the train set: {len(train_data)}. Size of the validation set: {len(valid_data)}")

    chars_per_token = chars_token_ratio(train_data, tokenizer)
    print(f"The character to token ratio of the dataset is: {chars_per_token:.2f}")

    train_dataset = ConstantLengthDataset(
        tokenizer,
        train_data,
        infinite=True,
        seq_length=seq_length,
        chars_per_token=chars_per_token,
    )
    valid_dataset = ConstantLengthDataset(
        tokenizer,
        valid_data,
        infinite=False,
        seq_length=seq_length,
        chars_per_token=chars_per_token,
    )
    return train_dataset, valid_dataset

Preparing Training

In [ ]:
model = LlamaForCausalLM.from_pretrained(model_path,trust_remote_code = True, load_in_8bit = True, device_map = 'auto')
model = prepare_model_for_int8_training(model)
target_modules = ["q_proj", "v_proj"]
lora_config = LoraConfig(
    r=16,
    lora_alpha = 32,
    target_modules = target_modules,
    lora_dropout = 0.05,
    bias = 'none',
    task_type = 'CAUSAL_LM'
)

model = get_peft_model(model,lora_config)

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [ ]:
print_trainable_parameters(model)

trainable_params8388608 || all_params:6746804224 || ration%:0.12433454005023165


In [ ]:
train_dataset,eval_dataset = create_datasets(tokenizer, dataset_name,subset,split,streaming,seed,size_valid_set,shuffle_buffer)

Loading the dataset in streaming mode


  0%|          | 0/400 [00:04<?, ?it/s]

The character to token ratio of the dataset is: 2.71


In [ ]:
os.makeirs(output_dir)

In [ ]:
 training_args = TrainingArguments(
        output_dir=output_dir,
        dataloader_drop_last=True,
        evaluation_strategy="steps",
        max_steps=max_steps,
        eval_steps=eval_freq,
        save_steps=save_freq,
        logging_steps=log_freq,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        learning_rate=learning_rate,
        lr_scheduler_type=lr_scheduler_type,
        warmup_steps=num_warmup_steps,
        gradient_accumulation_steps=gradient_accumulation_steps,
        bf16=bf16,
        weight_decay=weight_decay,
        run_name="llama-7b-finetuned",
        report_to="wandb",
        ddp_find_unused_parameters=False,
    )

In [ ]:
trainer = Trainer(model,args = training_args,train_dataset = train_dataset,eval_dataset=eval_dataset)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
100,1.696300,1.450831


Step,Training Loss,Validation Loss
100,1.696300,1.450831
200,1.157800,1.444253
300,1.796300,1.443699


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/transformers/trainer.py:1633 in train                     │
│                                                                                                  │
│   1630 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1631 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1632 │   │   )                                                                                 │
│ ❱ 1633 │   │   return inner_training_loop(                                                       │
│   1634 │   │   │   args=args,                                                                    │
│   1635 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1636 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/transformers/trainer.py:1979 in _inner_training_loop      │
│                                                                                                  │
│   1976 │   │   │   │   │   self.state.epoch = epoch + (step + 1 + steps_skipped) / steps_in_epo  │
│   1977 │   │   │   │   │   self.control = self.callback_handler.on_step_end(args, self.state, s  │
│   1978 │   │   │   │   │                                                                         │
│ ❱ 1979 │   │   │   │   │   self._maybe_log_save_evaluate(tr_loss, model, trial, epoch, ignore_k  │
│   1980 │   │   │   │   else:                                                                     │
│   1981 │   │   │   │   │   self.control = self.callback_handler.on_substep_end(args, self.state  │
│   1982                                                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/transformers/trainer.py:2236 in _maybe_log_save_evaluate  │
│                                                                                                  │
│   2233 │   │   │   │   │   │   metric_key_prefix=f"eval_{eval_dataset_name}",                    │
│   2234 │   │   │   │   │   )                                                                     │
│   2235 │   │   │   else:                                                                         │
│ ❱ 2236 │   │   │   │   metrics = self.evaluate(ignore_keys=ignore_keys_for_eval)                 │
│   2237 │   │   │   self._report_to_hp_search(trial, self.state.global_step, metrics)             │
│   2238 │   │                                                                                     │
│   2239 │   │   if self.control.should_save:                                                      │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/transformers/trainer.py:2931 in evaluate                  │
│                                                                                                  │
│   2928 │   │   start_time = time.time()                                                          │
│   2929 │   │                                                                                     │
│   2930 │   │   eval_loop = self.prediction_loop if self.args.use_legacy_prediction_loop else se  │
│ ❱ 2931 │   │   output = eval_loop(                                                               │
│   2932 │   │   │   eval_dataloader,                        

In [ ]:
peft_model_id = "LLAMA_peft_stack"

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid.
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as d

In [ ]:
model.push_to_hub(peft_model_id,use_auth_token = True)

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Kororinpa/LLAMA_peft_stack/commit/3c44189233d5ccec4bd9d3e49120fa83a459c516', commit_message='Upload model', commit_description='', oid='3c44189233d5ccec4bd9d3e49120fa83a459c516', pr_url=None, pr_revision=None, pr_num=None)